# Dutch SNOMED to concept table
This notebook describes how to create a SNOMED concept table containing Dutch names, to be used in a named entity recognition and linking tool such as MedCAT. 

In [ ]:
import json
import os
import re
import numpy as np
import pandas as pd
from pathlib import Path

# Input
nl_terms = Path('01_Download/SnomedCT_ManagedServiceNL_PRODUCTION_NL1000146_20220930T120000Z/Snapshot/Terminology/sct2_Description_Snapshot-nl_NL1000146_20220930.txt')

# Output
output_file_name = '04_ConceptDB/snomedct-dutch_v1.3.csv'
output_file_name_unfiltered = '04_ConceptDB/snomedct-dutch_v1.3-unfiltered.csv'

## Sources files

### SNOMED September 2020 release
| Edition | Total names | NL names | Description |
| - | - | - | - |
| Edition | 2263140 | 736619 | Include international SNOMED |
| Extension | 770016 | 736619 | Some terms are only in English |
| Patient Friendly | 1437 | 1437 | Small but potentially useful list of synonyms |

### SNOMED March 2021 release
| Edition | Total names | NL names | Description |
| - | - | - | - |
| Edition | 2422738 | 880806 | Include international SNOMED |
| Extension | 916553 | 880806 | Some terms are only in English |
| Patient Friendly | 2004 | 2004 | Small but potentially useful list of synonyms |

### SNOMED September 2021 release

| Edition | Total names | NL names | Description |
| - | - | - | - |
| Edition | 2469845 | 910228 | Include international SNOMED |
| Extension | 948571 | 910228 | Some terms are only in English |
| Patient Friendly | 2385 | 2385 | Small but potentially useful list of synonyms |


### SNOMED September 2022 release
To count total number of records in the SNOMED files, use `wc -l` and substract 1 for header.
```bash
wc -l SnomedCT_*/Snapshot/Terminology/sct2_Description_*.txt
```

| File | Language | NL names |
| - | - | - |
| sct2_Description_Snapshot-en_NL1000146_20220930.txt | English | 1611131 |
| sct2_Description_Snapshot-nl_NL1000146_20220930.txt | Dutch | 945292 |


### Description type
The SNOMED description table contains 3 types of records:

|Type id|Term|
|-|-|
|900000000000003001|Fully specified name (FSN)|
|900000000000013009|Synonym|
|900000000000550004|Definition|

The the purpose of creating a list of names for entity recognation, terms must be:
- FSN or Synonym
- Active

In [ ]:
def parse_file(filename):
    with open(filename, encoding='utf-8') as f:
        entities = [[n.strip() for n in line.split('\t')] for line in f]
        return pd.DataFrame(entities[1:], columns=entities[0])

In [ ]:
# Select active terms
df_terms = parse_file(nl_terms)
df_terms_active = df_terms.loc[df_terms.active == '1'].copy()
df_terms_inactive = df_terms.loc[df_terms.active != '1'].copy()
print(f'Active terms: {df_terms_active.shape[0]}')
print(f'Inactive terms: {df_terms_inactive.shape[0]}\n')

# Extract fully specified names
df_fsn = df_terms_active[(df_terms_active.typeId == '900000000000003001')].copy()
print(f'Active FSN: {df_fsn.shape[0]}')

# Extract synonyms
df_terms_synonyms = df_terms_active[(df_terms_active.typeId == '900000000000013009')].copy()
print(f'Active Synonyms: {df_terms_synonyms.shape[0]}\n')

### Primary concepts

In [ ]:
df_primary_concepts = df_fsn.copy()
df_primary_concepts['tui'] = df_primary_concepts['term'].str.extract(r"\(([^)]*)\)[^(]*$")
df_primary_concepts['str'] = df_primary_concepts['term'].str.extract(r"(^[^\(]+)")
df_primary_concepts['str'] = df_primary_concepts['str'].str.strip()
df_primary_concepts = df_primary_concepts[['conceptId', 'typeId', 'tui', 'str']]

# Create CUI-TUI mapping
cui_tui_mapping = dict(zip(df_primary_concepts.conceptId, df_primary_concepts.tui))
df_primary_concepts.head()

### Synonyms

In [ ]:
# Clean synonym table
df_synonyms = df_terms_synonyms.copy()
df_synonyms = df_synonyms[['conceptId', 'term', 'typeId']]
df_synonyms.rename({'term': 'str'}, inplace=True, axis=1)

# Add TUI to synonyms
df_synonyms['tui'] = df_synonyms.conceptId.map(cui_tui_mapping)
# df_synonyms[df_synonyms['tui'].isna()].shape
# 67 synonyms without type

df_synonyms.head()

### Combined

In [ ]:
# Create combined 
df_all = pd.concat([df_primary_concepts, df_synonyms]).reset_index(drop=True)
df_all.rename({'typeId': 'tty', 'conceptId': 'cui'}, inplace=True, axis=1)

# Map to MedCAT's P (Preferred term) & A values
# See https://github.com/CogStack/MedCAT/blob/master/examples/README.md
df_all.tty.replace({'900000000000003001': 'P',
                    '900000000000013009': 'A'}, inplace=True)

# Use convention in UMLS where default English SNOMED is called SNOMEDCT_US
df_all['sab'] = 'SNOMEDCT_NL'
df_all = df_all[['cui', 'str', 'tty', 'tui', 'sab']]

# Drop synonyms that are the same as primary name
print(f'Records before dropping duplicates: {df_all.shape[0]}')
df_all_unique = df_all.drop(df_all[(df_all.duplicated(subset=['cui', 'str', 'tui'], keep=False)) & (df_all.tty=='A')].index)
print(f'Records after dropping duplicates: {df_all_unique.shape[0]}')

# Sort column on cui and tty
df_all_unique['cui'] = df_all_unique['cui'].astype(int)
df_all_unique.sort_values(['cui', 'tty'], ascending=[True, False], inplace=True)

df_all_unique.head(25)

In [ ]:
# A few rows contain NaN
# Easiest way to deal with it is drop them. 
display(df_all_unique[df_all_unique.isnull().any(axis=1)].head())
print(len(df_all_unique))
df_all_unique.dropna(inplace=True)
print(len(df_all_unique))

### Examples

In [ ]:
df_all_unique[df_all_unique.str == 'ALS']

In [ ]:
df_all_unique[df_all_unique.cui == 86044005]

In [ ]:
df_all_unique[df_all_unique.str == 'longkanker']

In [ ]:
df_all_unique[df_all_unique.cui == 93880001]

In [ ]:
df_all_unique[df_all_unique.cui == 22298006]

In [ ]:
df_all_unique[df_all_unique.str == 'methotrexaat']

In [ ]:
df_all_unique[df_all_unique.cui == 387381009]

## Evaluation of SNOMED types

To select the types relevant for named entity linking, we assessed the performance of a MedCAT model on a set of example documents using the unfiltered SNOMED terms. We noticed some types are not useful for our general purpose (named entity recognition), and introduce false positives and ambiguity. We exclude the less useful types from our concept table.

In [ ]:
df_all_unique.tui.value_counts()

| tui | usefulness for NER | useful examples for NER | useless examples for NER |
| :- | :- | :- | :-|
|aandoening |good|hypertensie, boezemfibrilleren, av-blok| |
|monster|good|trombocyten, leukocyten,basofiele granulocyten||
|regime/therapie|good|fysiotherapie, hartrevalidatie, therapie||
|waarneembare entities|good|leeftijd, bloeddruk, hartas, LVEF||
|bevinding|good|koorts, zwelling, tachycardie||
|attribuut|good|bij, na||
|kwalificatiewaarde|good|ontslag, beloop, gestaakt, geen||
|afwijkende morfologie|good|thermisch letsel, blaar, luxatie||
|cel|good, but rare|erythrocytes||
|gegevensobject|good, but rare| Echocardiogram, operatieverslag||
|sociaal concept|good, but rare|familie||
|situatie|good, but includes negation|geen pijn, geen dispneu, geen hoesten||
|verrichting |decent|lokale anesthesie, lichamelijk onderzoek, palpatie|erg (elektroretinografie), weken, post (peritoneale transfer van eicel en sperma), beleid (management)|
|substantie|decent|nebivolol, amlodipine, ceftriaxon|wortel, PM (fijnstof)|
|omgeving|decent|ziekenhuis, polikliniek, huis, afdeling cardiologie, afdeling fysiologie|meer, stroop, plaats|
|lichaamsstructuur|decent|pols, aortaklep, AV-knoop|mid (mesioincisodistale vlakken van gebitselement)|
|persoon|decent|patient, dochter, vader|bekende (kennis)|
|fysische kracht|decent, but rare|druk||
|fysiek object|discutable, but rare|pacemaker|verband|
|beroep|bad||rechter, belang (behanger), herkende (werkende)|
|metadata|bad||beeld|
|gebeurtenis|bad, only 1 term matched||het (hoog energetisch trauma)|

Terms that were never found: omgeving/locatie, organisme, physical object: fout, religie/filosofie, product, disorder, navigatieconcept, lifestyle, procedure, gradering, tumorgradering, beoordelingsschaal, inactief concept, speciaal concept, ras, foundation metadata concept, physical object, eigenschap, celstructuur

In [ ]:
# Total number of concepts
df_all_unique.shape[0]

In [ ]:
# Number of primary concepts
df_all_unique[df_all_unique.tty == 'P'].shape

In [ ]:
# Number of synonyms
df_all_unique[df_all_unique.tty == 'A'].shape

In [ ]:
# Define list of relevant terms for general use
relevant_tuis = ['aandoening',
                'monster',
                'regime/therapie',
                'bevinding',
                'afwijkende morfologie',
                'cel',
                'gegevensobject',
                'verrichting',
                'substantie',
                'lichaamsstructuur']

In [ ]:
df_all_unique[df_all_unique.tui.isin(relevant_tuis)].tui.value_counts()

In [ ]:
df_all_unique[df_all_unique.tui.isin(relevant_tuis)].shape[0]

## Output

In [ ]:
# Unfiltered concepts
print(f'Number of concepts: {len(df_all_unique.cui.unique())}')
print(f'Number of names: {len(df_all_unique)}')
df_all_unique.to_csv(output_file_name_unfiltered, index=False)

In [ ]:
# Filtered concepts
filtered_concepts = df_all_unique[df_all_unique.tui.isin(relevant_tuis)]
print(f'Number of concepts: {len(filtered_concepts.cui.unique())}')
print(f'Number of names: {len(filtered_concepts)}')
filtered_concepts.to_csv(output_file_name, index=False)